# Momento de Retroalimentación 1: Módulo 2

### Ricardo Andres Arriaga Quezada A01570553

## Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

## Creacion de dataframes para regresion logistica

Se utilizara un dataset que describe el peso y la estatura de hombres y mujeres. La idea sera clasificar por sexo segun ambas variables, los sexos se clasificaran de la siguiente manera:
* Hombres = 0
* Mujeres = 1

In [6]:
data = pd.read_csv("Estatura-peso_HyM.csv")

dataH = data.loc[:, ["H_estat", "H_peso"]]
dataH.columns = ["Estatura", "Peso"]
dataM = data.loc[:, ["M_estat", "M_peso"]]
dataM.columns = ["Estatura", "Peso"]

# Identificadores de sexo: 0 = Hombre, 1 = Mujer
dataH["Sexo"] = 0
dataM["Sexo"] = 1

# Union de ambos dataframes
df = pd.concat([dataH, dataM])

sex = df["Sexo"].unique().tolist()
n = len(sex)

## Entrenamiento del modelo

In [20]:
X = df.drop(columns = ["Sexo"], axis = 1)
y = df.Sexo.values
X_train, X_test, y_train, y_test = train_test_split(X, y)

alpha = 0.000001
iters = 500
thetas = np.full((n, 3), 0.0001)
h = lambda x, theta, x1: 1/(1+m.exp(-(theta[0] + theta[1]*x + theta[2]*x1)))

n_train = len(y_train)

for k in range(n):
    for idx in range(iters):
        acumDelta = {
                    "x_" : [],
                    "x" : [],
                    "x1" : []
                    }
        for (i_row, X), y in zip(X_train.iterrows(), y_train):
            if y != (k+1):
                y = 0
            else:
                y = 1
                
            acumDelta['x_'].append(h(X.Estatura, thetas[k], X.Peso) - y)
            acumDelta['x'].append((h(X.Estatura, thetas[k], X.Peso) - y) * X.Estatura)
            acumDelta['x1'].append((h(X.Estatura, thetas[k], X.Peso) - y) * X.Peso)

        sJt_ = sum(acumDelta['x_'])
        sJt0 = sum(acumDelta['x'])
        sJt1 = sum(acumDelta['x1'])

        thetas[k][0] = thetas[k][0] - alpha/n_train*sJt_
        thetas[k][1] = thetas[k][1] - alpha/n_train*sJt0
        thetas[k][2] = thetas[k][2] - alpha/n_train*sJt1

thetas

array([[ 1.11234627e-04,  1.08469836e-04, -1.28574904e-03],
       [-9.80971069e-05, -2.18742152e-04, -1.24672003e-02]])

## Validacion de modelo

In [22]:
print("Valores Reales")
print(y_test)
print()

def round(n, decimals=0):
    multiplier = 10 ** decimals
    return m.floor(n*multiplier + 0.5) / multiplier

def predict(X, thetas):
    p = []
    for k in range(n):
        p.append((h(X.Estatura, thetas[k], X.Peso)))
    print(p)

predicts = []
for idx, value in X_test.iterrows():
    predicts.append(predict(value, thetas))

Valores Reales
[1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 1
 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1
 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0]

[0.48098564157993945, 0.3227879490073581]
[0.47751531835929, 0.2940287269464349]
[0.4856862211548777, 0.3639017704153682]
[0.4822260239230957, 0.333427054379329]
[0.47517651738583455, 0.27549337550568354]
[0.4810945023150224, 0.3237157178200097]
[0.48337567804310266, 0.3434189358947516]
[0.47568690965682897, 0.27946662737853584]
[0.48422609037705056, 0.35087311548447564]
[0.48192321762392604, 0.33079605979250526]
[0.4857892683395918, 0.36482524351247614]
[0.4862683777987509, 0.36913954157604684]
[0.47745170252987, 0.29351050781689275]
[0.4816974657389753, 0.3288356370636314]
[0.4763365725179696, 0.2845934126878312]
[0.48131919051954103, 0.32562920671416534]
[0.47772866578029816, 0.29573689045832313]
[0.4773418649870277, 0.2926024202300299]
[0.4769836

En esta primera instancia se puede observar que ningun valor pasa de 0.5, el cual es nuestro limite para declarar si la persona es hombre o mujer. Pero se puede observar que los valores se acercan mucho a una prediccion correcta si ser rendondean hacia arriba o abajo.

Realizaremos una prueba ajustando los resultados para probar si las predicciones son correctas.

In [23]:
def round(n, decimals=0):
    multiplier = 10 ** decimals
    return m.floor(n*multiplier + 0.5) / multiplier

def predict_Adj(X, thetas):
    p = []
    for k in range(n):
        p.append((h(X.Estatura, thetas[k], X.Peso) + 0.02))
    
    ypred = max(p)
    if((ypred + 0.5) >= 1.0):
        return 1
    else:
        return 0
    
predicts = []
for idx, value in X_test.iterrows():
    predicts.append(predict_Adj(value, thetas))

print("Valores Reales")
print(y_test)
print()
print("Valores Predecidos")
print(predicts)
acc = accuracy_score(predicts, y_test)
cm = confusion_matrix(predicts, y_test)
print()
print('Accuracy:', acc*100, '%')
print()
print("Matriz de Prediccion")
print(cm)

Valores Reales
[1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 1
 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1
 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0]

Valores Predecidos
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0]

Accuracy: 89.0909090909091 %

Matriz de Prediccion
[[54  7]
 [ 5 44]]


Sorpresivamente podemos ver que los resultados son precisos aunque cabe destacar que se realizaron distintas pruebas y el valor de la R^2 oscilo en un rango de 91.88% y 41.81%. El parametro de random_state, a la hora de crear las divisiones, es el responsable de crear estos cambios de precision ya que el modelo tendra que ajustarse a los resultados obtenidos y tomando en cuenta el pequeño margen de error que hay entre la clasificicacion de hombres y mujeres se producen dichos resultados. Podemos anlcar este valor para obtener mejores resultados, en este caso no estamos anclando el random_state por lo que cada vez que se corra obtemndremos valores diferentes.

## Predicciones
Ahora haremos pruebas de predicciones con valores fuera del dataset basados en la tabla del IMC.

In [35]:
predicciones = {"Estatura": [1.68, 1.68, 1.78, 1.78, 1.65, 1.65, 1.73, 1.73, 1.65, 1.65],
                "Peso": [66.21, 61.83, 73.36, 64.0, 58.33, 52.12, 66.63, 59.42, 64.87, 59.52],
                "Sexo":[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]}

prediccionesDF = pd.DataFrame(predicciones)

predicts = []
for idx, value in prediccionesDF.iterrows():
    predicts.append(predict_Adj(value, thetas))

print("Valores Reales")

for i in range(len(prediccionesDF.Sexo)):
    print(prediccionesDF.Sexo[i], end=" ")
print()
print("Valores Predecidos")
print(predicts)
acc = accuracy_score(predicts, prediccionesDF.Sexo)
cm = confusion_matrix(predicts, prediccionesDF.Sexo)
print()
print('Accuracy:', acc*100, '%')
print()
print("Matriz de Prediccion")
print(cm)

Valores Reales
0 1 0 1 0 1 0 1 0 1 
Valores Predecidos
[0, 1, 0, 0, 1, 1, 0, 1, 0, 1]

Accuracy: 80.0 %

Matriz de Prediccion
[[4 1]
 [1 4]]


## Conclusion

En conclusuion el modelo funciona pero sufre de no poder distinguir con mejor precision entre ambas clases lo que lleva a tener que realizar decisiones con muy poco margen de error. Esta es la mayor correccion que se le haria a este modelo.